<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/simulationTEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulation of a TEG dehdyration process
In this notebook we will establish code for using NeqSim for simulation of a TEG dehdyration process.

In [1]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *

UsageError: Line magic function `%%capture` not found.


# Seting up the process in Python code


In [2]:
#Input parameters
inputData = {
  "feedGasFlowRate": (117000.0+90000.0)*24.0/1.0e6,
  "feedGasTemperature": 30.0,
  "feedGasPressure": 44.0,
  "absorberFeedGasTemperature": 30.7,
  "absorberFeedGasPressure": 44.0,
  "leanTEGFlowRate": 5932.0*1.1,
  "leanTEGTemperature": 36.2,
  "flashDrumPressure": 6.0,
  "reboilerPressure": 1.3,
  "condenserTemperature": 94.0,
  "condenserPressure": 1.2,
  "reboilerTemperature": 201.0,
  "strippingGasRate": 300.0,
  "strippingGasFeedTemperature": 200.0,
  "regenerationGasCoolerTemperature": 35.0,
  "TEGpumpIsentropicEfficiency": 0.75,
  'finefilterdeltaP': 0.01,
  'carbonfilterdeltaP': 0.01,
  "numberOfEquilibriumStagesTEGabsorber": 4,
  "stageEfficiencyTEGabsorber": 0.53,
  "stageEfficiencyStripper": 0.5,
  "numberOfEquilibriumStagesStripper": 7,
  "UAvalueLeanRichTEGHeatExchanger": 16000.0
}

In [3]:
from neqsim.thermo import fluid, printFrame
from neqsim.process import getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle2, setpoint, calculator

clearProcess()

# Start by creating a fluid in neqsim
feedGas = fluid("cpa")  # create a fluid using the SRK-Eo
feedGas.addComponent("nitrogen", 1.0)
feedGas.addComponent("CO2", 2.0)
feedGas.addComponent("methane", 85.0)
feedGas.addComponent("ethane", 5.0)
feedGas.addComponent("propane", 3.0)
feedGas.addComponent("i-butane", 2.0)
feedGas.addComponent("n-butane", 2.0)
feedGas.addComponent("water", 0.0)
feedGas.addComponent("TEG", 0.0)
feedGas.setMixingRule(10)

dryFeedGas = stream(feedGas)
dryFeedGas.setName('dry feed gas')
dryFeedGas.setFlowRate(inputData['feedGasFlowRate'], 'MSm3/day')
dryFeedGas.setTemperature(inputData['feedGasTemperature'], 'C')
dryFeedGas.setPressure(inputData['feedGasPressure'], 'bara')

saturatedFeedGas = saturator(dryFeedGas)
saturatedFeedGas.setName("water saturator")

waterSaturatedFeedGas = stream(saturatedFeedGas.getOutStream())
waterSaturatedFeedGas.setName("water saturated feed gas")

feedTEG = feedGas.clone()
feedTEG.setMolarComposition([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015, 0.985])

feedTPsetterToAbsorber = heater(waterSaturatedFeedGas)
feedTPsetterToAbsorber.setOutPressure(inputData['absorberFeedGasPressure'], "bara")
feedTPsetterToAbsorber.setOutTemperature(inputData['absorberFeedGasTemperature'], "C")

feedToAbsorber = stream(feedTPsetterToAbsorber.getOutStream())
feedToAbsorber.setName("feed to TEG absorber")

TEGFeed = stream(feedTEG)
TEGFeed.setName('lean TEG to absorber')
TEGFeed.setFlowRate(inputData['leanTEGFlowRate'], 'kg/hr')
TEGFeed.setTemperature(inputData['leanTEGTemperature'], 'C')
TEGFeed.setPressure(inputData['absorberFeedGasPressure'], 'bara')

absorber = simpleTEGAbsorber()
absorber.setName("TEG absorber")
absorber.addGasInStream(feedToAbsorber)
absorber.addSolventInStream(TEGFeed)
absorber.setNumberOfStages(inputData['numberOfEquilibriumStagesTEGabsorber'])
absorber.setStageEfficiency(inputData['stageEfficiencyTEGabsorber'])
absorber.setInternalDiameter(1.7)

dehydratedGas = stream(absorber.getGasOutStream())
dehydratedGas.setName('dry gas from absorber')

richTEG = stream(absorber.getSolventOutStream())
richTEG.setName("rich TEG from absorber")

glycol_flash_valve = valve(richTEG)
glycol_flash_valve.setName("Rich TEG HP flash valve")
glycol_flash_valve.setOutletPressure(inputData['flashDrumPressure'])

richGLycolHeaterCondenser = heater(glycol_flash_valve.getOutStream())
richGLycolHeaterCondenser.setName("rich TEG preheater")


flashSep = separator(richGLycolHeaterCondenser.getOutStream())
flashSep.setName("degasing separator")

flashGas = stream(flashSep.getGasOutStream())
flashGas.setName("gas from degasing separator")

flashLiquid = stream(flashSep.getLiquidOutStream())
flashLiquid.setName("liquid from degasing separator")

fineFilter = filters(flashLiquid)
fineFilter.setName("TEG fine filter")
fineFilter.setDeltaP(inputData['finefilterdeltaP'], "bara")

heatEx = heatExchanger(fineFilter.getOutStream())
heatEx.setName("lean/rich TEG heat-exchanger")
heatEx.setGuessOutTemperature(273.15 + 130.0)
heatEx.setUAvalue(inputData['UAvalueLeanRichTEGHeatExchanger'])

glycol_flash_valve2 = valve(heatEx.getOutStream(0))
glycol_flash_valve2.setName("Rich TEG LP flash valve")
glycol_flash_valve2.setOutletPressure(inputData['reboilerPressure'])

stripGas = feedGas.clone()

strippingGas = stream(stripGas)
strippingGas.setName('stripGas')
strippingGas.setFlowRate(inputData['strippingGasRate'], "Sm3/hr")
strippingGas.setTemperature(inputData['strippingGasFeedTemperature'], "C")
strippingGas.setPressure(inputData['reboilerPressure'], "bara")

gasToReboiler = strippingGas.clone()
gasToReboiler.setName("gas to reboiler")

column = distillationColumn(1, True, True)
column.setName("TEG regeneration column")
column.addFeedStream(glycol_flash_valve2.getOutStream(), 0)
column.getReboiler().setOutTemperature(273.15 + inputData['reboilerTemperature'])
column.getCondenser().setOutTemperature(273.15 + inputData['condenserTemperature'])
column.getReboiler().addStream(gasToReboiler)
column.setTopPressure(inputData['condenserPressure'])
column.setBottomPressure(inputData['reboilerPressure'])

coolerRegenGas = heater(column.getGasOutStream())
coolerRegenGas.setName("regen gas cooler")
coolerRegenGas.setOutTemperature(273.15 + inputData['regenerationGasCoolerTemperature'])

sepregenGas = separator(coolerRegenGas.getOutStream())
sepregenGas.setName("regen gas separator");

gasToFlare = stream(sepregenGas.getGasOutStream())
gasToFlare.setName("gas to flare");

liquidToTrreatment = stream(sepregenGas.getLiquidOutStream())
liquidToTrreatment.setName("water to treatment")

stripper = waterStripperColumn("TEG stripper")
stripper.addSolventInStream(column.getLiquidOutStream())
stripper.addGasInStream(strippingGas)
stripper.setNumberOfStages(inputData['numberOfEquilibriumStagesStripper'])
stripper.setStageEfficiency(inputData['stageEfficiencyStripper'])

recycleGasFromStripper = recycle2("stripping gas recirc")
recycleGasFromStripper.addStream(stripper.getGasOutStream())
recycleGasFromStripper.setOutletStream(gasToReboiler)

heatEx.setFeedStream(1, stripper.getSolventOutStream())

coolerhOTteg3 = heater(heatEx.getOutStream(1))
coolerhOTteg3.setName("lean TEG cooler")
coolerhOTteg3.setOutTemperature(273.15 + inputData['leanTEGTemperature'])

hotLeanTEGPump2 = pump(coolerhOTteg3.getOutStream(), inputData['absorberFeedGasPressure'], "lean TEG HP pump")
hotLeanTEGPump2.setName("lean TEG HP pump")
hotLeanTEGPump2.setOutletPressure(inputData['absorberFeedGasPressure'])
hotLeanTEGPump2.setIsentropicEfficiency(inputData['TEGpumpIsentropicEfficiency'])

pumpHPPresSet = setpoint("HP pump set", hotLeanTEGPump2, "pressure", feedToAbsorber)

leanTEGtoabs = stream(hotLeanTEGPump2.getOutStream())
leanTEGtoabs.setName("lean TEG to absorber")

pureTEG = feedGas.clone()
pureTEG.setMolarComposition([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])

makeupTEG = stream(pureTEG)
makeupTEG.setName("makeup TEG")
makeupTEG.setFlowRate(1e-6, "kg/hr")
makeupTEG.setTemperature(inputData['leanTEGTemperature'], "C")
makeupTEG.setPressure(inputData['absorberFeedGasPressure'], "bara")

makeupCalculator = calculator("TEG makeup calculator")
makeupCalculator.addInputVariable(dehydratedGas)
makeupCalculator.addInputVariable(flashGas)
makeupCalculator.addInputVariable(gasToFlare)
makeupCalculator.addInputVariable(liquidToTrreatment)
makeupCalculator.setOutputVariable(makeupTEG)

makeupMixer = mixer("makeup mixer")
makeupMixer.addStream(leanTEGtoabs)
makeupMixer.addStream(makeupTEG)

resycleLeanTEG = recycle2("lean TEG resycle")
resycleLeanTEG.addStream(makeupMixer.getOutStream())
resycleLeanTEG.setOutletStream(TEGFeed)
resycleLeanTEG.setPriority(200)
resycleLeanTEG.setDownstreamProperty("flow rate")

richGLycolHeaterCondenser.setEnergyStream(column.getCondenser().getEnergyStream())

TEGprocess = getProcess()


# Run simulation

In [30]:
TEGprocess.run()

# Read results from simulation

In [26]:
wetGasRatekghr = TEGprocess.getUnit("water saturated feed gas").getFlowRate("kg/hr")
waterInWetGasppm = TEGprocess.getUnit("water saturated feed gas").getFluid().getPhase(0).getComponent('water').getz()*1.0e6
waterInDryGasppm = TEGprocess.getUnit("dry gas from absorber").getFluid().getPhase(0).getComponent('water').getz()*1.0e6
waterInWetGaskgMSm3 = waterInWetGasppm*0.01802*101325.0/(8.314*288.15)
waterInDryGaskgMSm3 = waterInDryGasppm*0.01802*101325.0/(8.314*288.15)
waterInWetGaskghr = waterInWetGaskgMSm3*TEGprocess.getUnit("water saturated feed gas").getFlowRate("Sm3/day")/1.0e6/24.0
waterInDryGaskghr = waterInDryGaskgMSm3*TEGprocess.getUnit("dry gas from absorber").getFlowRate("Sm3/day")/1.0e6/24.0
waterInDryGaskgMSm3 = waterInDryGasppm*0.01802*101325.0/(8.314*288.15)
wtLeanTEGFromReboiler = TEGprocess.getUnit("TEG regeneration column").getLiquidOutStream().getFluid().getPhase("aqueous").getWtFrac("TEG")*100.0
wtLeanTEGFromStripper = TEGprocess.getUnit("TEG stripper").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("TEG")*100.0
wtWaterFromStripper = TEGprocess.getUnit("TEG stripper").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("water")*100.0
wtRichTEGFromAbsorber = TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("TEG")*100.0
richTEGtemperature = TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFluid().getTemperature("C")
waterInRichTEGkghr = TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("water")*TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFluid().getFlowRate("kg/hr")
TEGcirculationratelitreperkg = TEGprocess.getUnit("TEG absorber").getSolventInStream().getFlowRate("kg/hr")*wtLeanTEGFromStripper/100/waterInWetGaskghr
richTEGtemperatureAfterDepres = TEGprocess.getUnit("Rich TEG HP flash valve").getOutStream().getFluid().getTemperature("C")
RichTEGwtprbinary = TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("TEG")/(TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("TEG")+TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFluid().getPhase("aqueous").getWtFrac("water"))*100
reboilerdutykW = TEGprocess.getUnit("TEG regeneration column").getReboiler().getDuty()/1.0e3
condenserdutykW = TEGprocess.getUnit("TEG regeneration column").getCondenser().getDuty()/1.0e3
condensertemperature= TEGprocess.getUnit("TEG regeneration column").getCondenser().getTemperature()-273.15
reboilertemperature= TEGprocess.getUnit("TEG regeneration column").getReboiler().getTemperature()-273.15
TEGmakeupkghr = TEGprocess.getUnit("TEG makeup calculator").getOutputVariable().getFluid().getFlowRate("kg/hr")
TEGmakeupkgMSm3 = TEGmakeupkghr/TEGprocess.getUnit("dry feed gas").getFlowRate("Sm3/day")/1.0e6*24
leanTEGflow = TEGprocess.getUnit("TEG absorber").getSolventInStream().getFlowRate("kg/hr")
richTEGflow = TEGprocess.getUnit("TEG absorber").getSolventOutStream().getFlowRate("kg/hr")
pump2DutykW = TEGprocess.getUnit("lean TEG HP pump").getEnergy()/1.0e3
flashGasRate = TEGprocess.getUnit("gas from degasing separator").getFlowRate("kg/hr")
gasToFLareRatekghr = TEGprocess.getUnit("gas to flare").getFlowRate("kg/hr")
stripGasRatekghr = TEGprocess.getUnit("stripGas").getFlowRate("kg/hr")
waterToTreatment = TEGprocess.getUnit("water to treatment").getFlowRate("kg/hr")
TEGinwatertoTreatmentwtprecent = TEGprocess.getUnit("water to treatment").getFluid().getPhase("aqueous").getWtFrac("TEG")*100.0
flowFromReboiler = TEGprocess.getUnit("TEG regeneration column").getReboiler().getLiquidOutStream().getFlowRate("kg/hr")
richTEGpreheaterDutykW = TEGprocess.getUnit("rich TEG preheater").getDuty()/1000.0
richTEGpreheaterTemperature = TEGprocess.getUnit("rich TEG preheater").getOutStream().getTemperature("C")
richTEGtemperatureOutOfLPflashValve = TEGprocess.getUnit("Rich TEG LP flash valve").getOutStream().getTemperature("C")
waterInGasFromCondenser= TEGprocess.getUnit("TEG regeneration column").getCondenser().getGasOutStream().getFlowRate("kg/hr")*TEGprocess.getUnit("TEG regeneration column").getCondenser().getGasOutStream().getFluid().getPhase(0).getWtFrac("water")
richTEGtemperatureIntoHX1 = TEGprocess.getUnit("lean/rich TEG heat-exchanger").getOutStream(0).getTemperature("C")
richTEGtemperatureOutHX1 = TEGprocess.getUnit("lean/rich TEG heat-exchanger").getOutStream(0).getTemperature("C")
leanTEGtemperatureintoHX1  = TEGprocess.getUnit("lean/rich TEG heat-exchanger").getOutStream(1).getTemperature("C")   
leanTEGtemperatureoutTEGHX1 =  TEGprocess.getUnit("lean/rich TEG heat-exchanger").getOutStream(1).getTemperature("C")


# Update and run simulation

In [12]:
dryFeedGas.setTemperature(35.0, 'C')
TEGprocess.run()
waterInDryGasppm = TEGprocess.getUnit("dry gas from absorber").getFluid().getPhase(0).getComponent('water').getz()*1.0e6

In [29]:
waterInWetGasppm

1476.5837081998125